Loading the data

id: household

Choices:  
- heinz41  
- heinz32  
- heinz28  
- hunts32  

variables  
- disp (1/0)  
- feat (1/0)  
- price (Log?)  

In [29]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
df = pd.read_pickle("data.pkl") 
df.head()

,id,disp.heinz41,disp.heinz32,disp.heinz28,disp.hunts32,feat.heinz41,feat.heinz32,feat.heinz28,feat.hunts32,price.heinz41,price.heinz32,price.heinz28,price.hunts32,choice,choiceindex
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28,2
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz28,2
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz28,2
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz28,2
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz28,2


Generating data to benchmark

In [45]:
%%time
#brand choiced

### Configuration ###########

#number of datasets
n = 1

# True coefficients
c = [3.0, #alpha heinz41
     2.0, #alpha heinz32 
     4.0, #alpha heinz28
     1.7, #mu    display
     2.1, #mu    feat
     1.2, #mu    price
     3.4, #sigma dispay 
     2.5, #sigma feat 
     4.2, #sigma price 
    ]

#############################

choices = ["heinz41", "heinz32", "heinz28", "hunts32"]

#defining the choice making process
def makechoice(row):
    utilities = np.zeros(4)
    
    for k in range(4):
        utility = 0; #start with alpha
        if k < 3: utility = c[k]
        
        utilities[k] = utility

    row.choiceindex = np.argmax(utilities)
    row.choice = choices[row.choiceindex]
    return row


#generating the dataset with the dgp
data = [df.head().copy().apply(makechoice, axis=1) for i in range(n)]


data[0].head()

Wall time: 5.99 ms


In [44]:
data[0]

,id,disp.heinz41,disp.heinz32,disp.heinz28,disp.hunts32,feat.heinz41,feat.heinz32,feat.heinz28,feat.hunts32,price.heinz41,price.heinz32,price.heinz28,price.hunts32,choice,choiceindex
0,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz32,1
1,1,0,0,0,0,0,0,0,0,4.6,4.3,5.2,4.4,heinz32,1
2,1,0,0,0,0,0,1,0,0,4.6,2.5,4.6,4.8,heinz32,1
3,1,0,0,0,0,0,0,0,0,4.6,3.7,5.2,3.4,heinz32,1
4,1,0,0,0,0,0,0,1,0,4.6,3.0,4.6,4.8,heinz32,1


1